Data Science Capstone Project Week 3 

Download and import necessaary libraries

In [9]:
# import needed libraries
import requests
#from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import pandas as pd
import csv
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 


Foursquare Credentials

In [10]:
CLIENT_ID = 'OEKMEBXITJT02VBZB3LP4ATYZ35OI3PLISMSZUTSJSFMBZQI' # your Foursquare ID
CLIENT_SECRET = 'POCWPILITAHTRCVJB1EKV1Y1OM30PZXOGVYAPZ4CCPZKHMUQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OEKMEBXITJT02VBZB3LP4ATYZ35OI3PLISMSZUTSJSFMBZQI
CLIENT_SECRET:POCWPILITAHTRCVJB1EKV1Y1OM30PZXOGVYAPZ4CCPZKHMUQ


In [28]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1008658788"
#url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_content = requests.get(url).text
soup = bs(html_content, "lxml")


In [30]:
table = soup.find("table", attrs={"class": "wikitable"})
#print(soup)
# identify columns
th = table.find_all("th")
headers = []
for x in th:
    headers.append(x.text.strip())

# identify rows  
tr = table.find_all("tr")
rows = []

# split elements into lists and remove empty elements
for x in tr[1:]:
    rows.append(x.text.split("\n"))
for x in rows:
    for y in x:
        x.remove('')

In [31]:
df = pd.DataFrame(rows, columns = headers) 
df = df[df.Borough != 'Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [32]:
df.shape

(103, 3)

In [35]:
#Download and read the csv file containing coordinates 
url = "https://cocl.us/Geospatial_data"
coords = pd.read_csv(url)

In [36]:
#Merge coordinates with boroughs dataframe
df.merge(coords, on="Postal Code")

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [40]:
print(neighborhood_latitude=df['latitude'[0]] )

KeyError: 'l'

In [38]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL

neighborhood_latitude=df['Latitude'[0]] 
neighborhood_longitude=df['Longitude'[0]]

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

KeyError: 'L'